In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import os

from nba_api.stats.static import players
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

from utils import combine_team_games

In [2]:
# Mapping Table for IDs -> Names

nba_players = players.get_players()

nba_players = pd.DataFrame(nba_players)
nba_players = nba_players.loc[nba_players.is_active == True]
nba_players = nba_players[['id', 'full_name']]

nba_players.head()

,id,full_name
10,1630173,Precious Achiuwa
22,203500,Steven Adams
24,1628389,Bam Adebayo
29,1630534,Ochai Agbaji
36,1631231,James Akinjo


In [3]:
# Mapping Table for IDs -> Teams

nba_teams = teams.get_teams()
nba_teams = pd.DataFrame(nba_teams)

nba_teams.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [92]:
# Get All Games

min_date = '2010-01-01'

nba_team_ids = list(nba_teams.id)
all_games = pd.DataFrame()

for id in nba_team_ids:
    
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = id)
    tmp = gamefinder.get_data_frames()[0]
    tmp = tmp.loc[tmp.GAME_DATE >= min_date]

    all_games = pd.concat([all_games, tmp])


# Manual adjustment for NBA Cup for subsequent combine_team_games() to work properly
all_games.loc[(all_games.GAME_ID == '0062400001') & (all_games.TEAM_ABBREVIATION == 'MIL'), 'MATCHUP'] = 'MIL vs. OKC'

all_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612737,ATL,Atlanta Hawks,0022400477,2025-01-03,ATL @ LAL,L,240,102,...,0.778,13.0,26.0,39.0,22,10.0,3,8,22,-17.0
1,22024,1610612737,ATL,Atlanta Hawks,0022400461,2025-01-01,ATL @ DEN,L,240,120,...,0.667,12.0,32.0,44.0,34,10.0,5,13,16,-19.0
2,22024,1610612737,ATL,Atlanta Hawks,0022400438,2024-12-29,ATL @ TOR,W,240,136,...,0.743,14.0,26.0,40.0,30,22.0,8,14,19,29.0
3,22024,1610612737,ATL,Atlanta Hawks,0022400427,2024-12-28,ATL vs. MIA,W,240,120,...,0.773,9.0,38.0,47.0,34,7.0,2,11,16,10.0
4,22024,1610612737,ATL,Atlanta Hawks,0022400413,2024-12-26,ATL vs. CHI,W,240,141,...,0.941,12.0,32.0,44.0,32,9.0,5,12,13,8.0


In [99]:
# Get single line game stats via combine_team_games()
# _H represents stats relating to the home team, _A reptesents away team

all_games_comb = combine_team_games(all_games)


# Create season_short

all_games_comb['SEASON_SHORT'] = all_games_comb.SEASON_ID.str[-4:]


# Get game types

game_type_dict = {'1': 'preseason', 
            '2': 'regular', 
            '4': 'playoffs', 
            '5': 'play-in', 
            '6': 'nba_cup'}

all_games_comb['GAME_TYPE'] = all_games_comb.SEASON_ID.str[:1].map(game_type_dict)


all_games_comb.head()

,SEASON_ID,TEAM_ID_H,TEAM_ABBREVIATION_H,TEAM_NAME_H,GAME_ID,GAME_DATE,MATCHUP_H,WL_H,MIN_H,PTS_H,...,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,SEASON_SHORT,GAME_TYPE
2,22024,1610612747,LAL,Los Angeles Lakers,0022400477,2025-01-03,LAL vs. ATL,W,240,119,...,26.0,39.0,22,10.0,3,8,22,-17.0,2024,regular
6,22024,1610612743,DEN,Denver Nuggets,0022400461,2025-01-01,DEN vs. ATL,W,240,139,...,32.0,44.0,34,10.0,5,13,16,-19.0,2024,regular
10,22024,1610612761,TOR,Toronto Raptors,0022400438,2024-12-29,TOR vs. ATL,L,240,107,...,26.0,40.0,30,22.0,8,14,19,29.0,2024,regular
13,22024,1610612737,ATL,Atlanta Hawks,0022400427,2024-12-28,ATL vs. MIA,W,240,120,...,34.0,42.0,23,9.0,4,8,19,-10.0,2024,regular
17,22024,1610612737,ATL,Atlanta Hawks,0022400413,2024-12-26,ATL vs. CHI,W,240,141,...,27.0,36.0,36,8.0,1,14,26,-8.0,2024,regular


In [113]:
#all_games_comb.loc[((all_games_comb.TEAM_ABBREVIATION_H == 'DEN') | (all_games_comb.TEAM_ABBREVIATION_A == 'DEN')) & (all_games_comb.SEASON_ID == '22016')]
all_games_comb.loc[(all_games_comb.SEASON_ID == '22011')].TEAM_NAME_H.value_counts()


Atlanta Hawks             33
Toronto Raptors           33
Memphis Grizzlies         33
Dallas Mavericks          33
Oklahoma City Thunder     33
Golden State Warriors     33
Minnesota Timberwolves    33
Houston Rockets           33
New Jersey Nets           33
San Antonio Spurs         33
New Orleans Hornets       33
Los Angeles Lakers        33
Phoenix Suns              33
Portland Trail Blazers    33
Chicago Bulls             33
New York Knicks           33
Indiana Pacers            33
Miami Heat                33
Detroit Pistons           33
Sacramento Kings          33
Denver Nuggets            33
Los Angeles Clippers      33
Cleveland Cavaliers       33
Washington Wizards        33
Milwaukee Bucks           33
Philadelphia 76ers        33
Charlotte Bobcats         33
Boston Celtics            33
Orlando Magic             33
Utah Jazz                 33
Name: TEAM_NAME_H, dtype: int64

In [114]:
all_games_comb.loc[(all_games_comb.TEAM_ABBREVIATION_H == 'LAL') & (all_games_comb.SEASON_ID == '22011')].sort_values(by='GAME_DATE')

,SEASON_ID,TEAM_ID_H,TEAM_ABBREVIATION_H,TEAM_NAME_H,GAME_ID,GAME_DATE,MATCHUP_H,WL_H,MIN_H,PTS_H,...,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,SEASON_SHORT,GAME_TYPE
26155,22011,1610612747,LAL,Los Angeles Lakers,0021100003,2011-12-25,LAL vs. CHI,L,240,87,...,25.0,41.0,21,14.0,4,14,21,1.0,2011,regular
52424,22011,1610612747,LAL,Los Angeles Lakers,0021100022,2011-12-27,LAL vs. UTA,W,239,96,...,28.0,46.0,20,6.0,6,10,28,-25.0,2011,regular
52420,22011,1610612747,LAL,Los Angeles Lakers,0021100038,2011-12-29,LAL vs. NYK,W,241,99,...,23.0,32.0,15,11.0,6,12,20,-17.0,2011,regular
35992,22011,1610612747,LAL,Los Angeles Lakers,0021100050,2011-12-31,LAL vs. DEN,W,240,92,...,28.0,36.0,22,11.0,5,14,17,-3.0,2011,regular
44893,22011,1610612747,LAL,Los Angeles Lakers,0021100081,2012-01-03,LAL vs. HOU,W,239,108,...,28.0,38.0,27,8.0,0,8,25,-9.0,2011,regular
40703,22011,1610612747,LAL,Los Angeles Lakers,0021100109,2012-01-06,LAL vs. GSW,W,239,97,...,24.0,34.0,19,9.0,1,18,28,-7.0,2011,regular
52412,22011,1610612747,LAL,Los Angeles Lakers,0021100125,2012-01-08,LAL vs. MEM,W,240,90,...,25.0,34.0,23,18.0,4,9,18,-8.0,2011,regular
52408,22011,1610612747,LAL,Los Angeles Lakers,0021100141,2012-01-10,LAL vs. PHX,W,240,99,...,26.0,35.0,18,9.0,1,11,16,-16.0,2011,regular
16329,22011,1610612747,LAL,Los Angeles Lakers,0021100169,2012-01-13,LAL vs. CLE,W,240,97,...,31.0,44.0,20,5.0,0,18,24,-5.0,2011,regular
31066,22011,1610612747,LAL,Los Angeles Lakers,0021100194,2012-01-16,LAL vs. DAL,W,240,73,...,38.0,49.0,16,7.0,5,15,15,-3.0,2011,regular


In [101]:
all_games_comb[['SEASON_SHORT', 'GAME_TYPE']].groupby(['SEASON_SHORT', 'GAME_TYPE']).value_counts()

SEASON_SHORT  GAME_TYPE
2009          playoffs       82
              regular       757
2010          playoffs       81
              preseason     111
              regular      1303
2011          playoffs       84
              preseason      30
              regular       990
2012          playoffs       85
              preseason     106
              regular      1304
2013          playoffs       89
              preseason     108
              regular      1311
2014          playoffs       81
              preseason     107
              regular      1316
2015          playoffs       86
              preseason     100
              regular      1313
2016          playoffs       79
              preseason      97
              regular      1313
2017          playoffs       82
              preseason      71
              regular      1323
2018          playoffs       82
              preseason      66
              regular      1324
2019          play-in         1
              pl

In [82]:
all_games_comb.loc[all_games_comb.SEASON_ID.str[:1] == '6']

,SEASON_ID,TEAM_ID_H,TEAM_ABBREVIATION_H,TEAM_NAME_H,GAME_ID,GAME_DATE,MATCHUP_H,WL_H,MIN_H,PTS_H,...,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A
49519,62023,1610612747,LAL,Los Angeles Lakers,0062300001,2023-12-09,LAL vs. IND,W,242,123,...,0.879,9.0,23.0,32.0,27,10.0,7,9,33,-14.0


#### 3 Point Attempts Per Game

In [29]:
all_games_comb[['SEASON_ID']].drop_duplicates().loc[all_games_comb.SEASON_ID.str[-4:] == '2017']

,SEASON_ID
2411,22017
2739,12017
8355,42017


In [60]:
all_games_comb[['GAME_DATE', 'SEASON_ID']].loc[all_games_comb.SEASON_ID == '62023'].value_counts().reset_index().sort_values(by='GAME_DATE')

,GAME_DATE,SEASON_ID,0
0,2023-12-09,62023,1


In [18]:
all_games_comb.loc[(all_games_comb.TEAM_ABBREVIATION_H == 'DEN') | (all_games_comb.TEAM_ABBREVIATION_A == 'DEN')].sort_values(by='GAME_DATE', ascending=False).head(10)

2        2025
6        2025
10       2024
13       2024
17       2024
         ... 
86243    2011
86248    2010
86251    2010
86256    2010
86260    2010
Name: GAME_DATE, Length: 21511, dtype: int64